In [1]:
import pandas as pd
import zipfile
import os

# Caminho onde estão os arquivos .zip
caminho = 'dados/GTFS'

# Lista dos arquivos GTFS
arquivos_zip = [f for f in os.listdir(caminho) if f.endswith('.zip')]


In [2]:
# Lista de arquivos GTFS que queremos processar
gtfs_files = [
    "agency.txt", "calendar.txt", "fare_attributes.txt", "fare_rules.txt",
    "frequencies.txt", "routes.txt", "shapes.txt", "stop_times.txt",
    "stops.txt", "trips.txt"
]

# Dicionário para acumular DataFrames por tabela
dfs_acumulados = {arquivo.replace(".txt", ""): [] for arquivo in gtfs_files}

# Iterar sobre cada arquivo .zip
for arquivo in arquivos_zip:
    data_ref = arquivo.replace("google_transit_", "").replace(".zip", "")
    caminho_arquivo = os.path.join(caminho, arquivo)

    with zipfile.ZipFile(caminho_arquivo, "r") as z:
        for gtfs_file in gtfs_files:
            if gtfs_file in z.namelist():
                df = pd.read_csv(z.open(gtfs_file))
                df["data_referencia"] = data_ref
                dfs_acumulados[gtfs_file.replace(".txt", "")].append(df)

# Concatenar todos os DataFrames e salvar em parquet
os.makedirs("dados", exist_ok=True)
for nome, lista_dfs in dfs_acumulados.items():
    if lista_dfs:  # só concatena se houver dados
        final_df = pd.concat(lista_dfs, ignore_index=True)
        final_df.to_parquet(f"dados/{nome}_historico.parquet", index=False)

In [4]:
# Carregar dataframes já concatenados
trips_df = pd.read_parquet("dados/trips_historico.parquet")
routes_df = pd.read_parquet("dados/routes_historico.parquet")
stop_times_df = pd.read_parquet("dados/stop_times_historico.parquet")
stops_df = pd.read_parquet("dados/stops_historico.parquet")

# Limpeza e ajustes
stop_times_df = stop_times_df[['trip_id', 'stop_id', 'stop_sequence', 'data_referencia']].drop_duplicates()
trips_df['route_id'] = trips_df['route_id'].str.replace("-", "")
routes_df['route_id'] = routes_df['route_id'].str.replace("-", "")

services = {
    'USD': "Todos", 'U__': "Úteis", 'US_': "Úteis e Sábados",
    '_SD': "Sábados e Domingos", '__D': "Domingos", '_S_': "Sábados"
}
trips_df['service_id'] = trips_df['service_id'].map(services)

# Merge final
trips_df = trips_df.merge(stop_times_df, on=['trip_id', 'data_referencia']).merge(stops_df, on=['stop_id', 'data_referencia'])

# Salvar versões tratadas
routes_df.to_parquet('dados/routes_historico.parquet', index=False)
trips_df.to_parquet('dados/trips_historico.parquet', index=False)